## Interactive Plotting with Plot.ly ##

In [ ]:
import os
import datetime

AM_NAME = "AM5"
FILE_PATH = AM_NAME + "_Clips/"
FIRST_STAMP = "00:00:00 06/13/2019"
TITLE = "Bird Vocalizations from 00:00 6/13/19 - 23:50 6/17/19 - "+AM_NAME

# get all file names in FILE_PATH
(_, _, clip_names) = next(os.walk(FILE_PATH))

# collect data
global_scores = []
with open("global_scores_"+AM_NAME+".txt", "r") as f:
    for line in f:
        global_scores.append(float(line.strip()))

# create range of datetime objects
base = datetime.datetime.strptime(FIRST_STAMP, "%H:%M:%S %m/%d/%Y")
time_stamps = [base + datetime.timedelta(minutes=x*10) for x in range(len(global_scores))]

In [ ]:
"""
For reference: https://plotly.com/python/line-and-scatter/
Plot an interacitve line graph
:param clip_names: 
:param x_data: 
:param y_data: 
:param xlabel: Label of x axis
:param ylabel: Label of y axis
:param plot_title: Title of the graph

:return: Will display an interactive graph
"""
import plotly.graph_objects as go
import numpy as np
import os
from IPython.display import Audio
from scipy.io import wavfile
import sounddevice as sd

def interactive_plot(clip_names, x_data, y_data, xlabel, ylabel, plot_title):
    fig = go.FigureWidget([go.Scatter(x=x_data,
                                      y=y_data,
                                      mode='lines+markers',
                                      # marker_color=red for < 0.5, green otherwise
                                      text=np.array(clip_names))])
                                      
    fig.update_layout(title=plot_title,
                      xaxis=dict(title=xlabel),
                      yaxis=dict(title=ylabel))
                      
    scatter = fig.data[0]
    colors = ['#a3a7e4'] * len(clip_names)
    scatter.marker.color = colors
    scatter.marker.size = [8] * 720
    fig.layout.hovermode = 'closest'
    
    def play_clip(trace, points, state):
#         Audio(audio_clip, autoplay=True)
        audio_idx = points.point_inds[-1]
        audio_clip = os.path.join(FILE_PATH + clip_names[audio_idx])
        
        rate, data = wavfile.read(audio_clip)
        sd.play(data, rate)
        
        c = list(scatter.marker.color)
        s = list(scatter.marker.size)
        for i in points.point_inds:
            c[i] = '#bae2be'
            s[i] = 9
            with fig.batch_update():
                scatter.marker.color = c
                scatter.marker.size = s
    
    scatter.on_click(play_clip)
    
    return fig

In [ ]:
# run interactive plot
fig = interactive_plot(clip_names, time_stamps, global_scores, "Time", "Prediction scores", TITLE)
fig